# Load line values

This is a demo that queries a geojson line file and evaluates forecasted near-term fire risk based on the [Wildfire Forecast](https://data.spatiafi.com/dataset/36) dataset.  We start by loading our App Credentials, and then using the [geojson statistics](https://docs.spatiafi.com/api/#operation/geojson_statistics_api_statistics_post) endpoint to fetch fire risk values near the line of interest, including a buffer of 50km.

---

Install `spatiafi` (if not already installed):

In [ ]:
!sudo apt -y install libgeos-dev

In [ ]:
%pip install --upgrade spatiafi cartopy xarray geopandas folium google-cloud-storage

In [1]:
import json

import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import spatiafi
from cartopy import crs as ccrs
from cartopy.feature import NaturalEarthFeature
from google.cloud import storage
from shapely.geometry import MultiLineString

In [2]:
session = spatiafi.get_session()
session

## Loading geojson line

In [3]:
storage_client = storage.Client()

bucket_name = "ce-datasets"
folder_name = "ce-cn/cn-rail-assets-vector/public/cn_rail_from_ntad"
file_name = "cn_rail_network_from_NTAD.geojson"

bucket = storage_client.bucket(bucket_name)
folder_path = f"{folder_name}/" if folder_name else ""
file_path = f"{folder_path}{file_name}"

blob = bucket.blob(file_path)

if blob.exists():
    blob_content = blob.download_as_text()
    geojson_objects = blob_content.splitlines()

    desired_object_id = 557
    desired_geojson_object = None

    for obj in geojson_objects:
        data = json.loads(obj)
        if data["properties"]["OBJECTID"] == desired_object_id:
            desired_geojson_object = data
            break
    else:
        print(f"GeoJSON object with OBJECTID = {desired_object_id} not found")
else:
    print("GeoJSON file not found in GCS")

In [4]:
# Extract the geometries from the GeoJSON dictionary
geometries = desired_geojson_object["geometry"]

# Convert the geometries to Shapely objects
shapely_geometries = MultiLineString(geometries["coordinates"])

# Create a GeoDataFrame from the Shapely geometries
gdf = gpd.GeoDataFrame(geometry=[shapely_geometries])

We show here an example of loading in a section of CN rail line. Once we have loaded in the geojson line, we add a buffer of 50km in order to see if there are locations near the rail line at risk of fire.
The following map is interactive and displays the CN rail section as well as the buffer.

In [5]:
# Creating an interactive map which is zoomable using Folium

def generate_buffer_meter(data, radius, geometry="geometry", crs="epsg:4326"):
    data = gpd.GeoDataFrame(data, geometry=geometry, crs=crs)
    data = data.to_crs("+proj=aeqd +units=m  +x_0=0 +y_0=0")
    data[geometry] = data[geometry].buffer(radius)
    data = data.to_crs(crs)
    return data

buffer_radius_meters = 50000

# Generate the buffered GeoDataFrame
buffered = generate_buffer_meter(gdf, buffer_radius_meters)

lons = buffered["geometry"][0].exterior.coords.xy[0]
lats = buffered["geometry"][0].exterior.coords.xy[1]
lon_lat_pairs = list(zip(lons, lats))

# Create a Folium map centered at a specific location
m = folium.Map(location=[45, -75], zoom_start=4)

# Convert the original GeoDataFrame to GeoJSON
original_geojson = gdf.to_json()

# Add the original geometries to the Folium map
folium.GeoJson(original_geojson, style_function=lambda x: {'color': 'blue'}).add_to(m)

# Convert the buffered GeoDataFrame to GeoJSON
buffered_geojson = buffered.to_json()

# Add the buffered geometries to the Folium map
folium.GeoJson(buffered_geojson, style_function=lambda x: {'fillColor': 'red'}).add_to(m)

# Display the map
m

In [6]:
url = "https://api.spatiafi.com/api/statistics"

params = {
    "item_id": "ce-cfsv2-era5-fwi-forecast-day01",
    "bidx": "1",
}

payload = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [lon_lat_pairs],
        "type": "Polygon",
    },
}

# `POST` the request using our `session` object, which will automatically handle authentication.
response = session.post(url, json=payload, params=params)
data = response.json()

print(data["properties"])

{'statistics': {'b1': {'min': 0.0, 'max': 14.412392616271973, 'mean': 6.470380769987128, 'count': 874.0, 'sum': 5655.11279296875, 'std': 5.144995033778678, 'median': 9.010396003723145, 'majority': 0.0, 'minority': 8.270759582519531, 'unique': 531.0, 'histogram': [[330.0, 0.0, 0.0, 0.0, 0.0, 41.0, 197.0, 191.0, 98.0, 17.0], [0.0, 1.4412392377853394, 2.8824784755706787, 4.3237175941467285, 5.764956951141357, 7.206196308135986, 8.647435188293457, 10.088674545288086, 11.529913902282715, 12.971153259277344, 14.412392616271973]], 'valid_percent': 100.0, 'masked_pixels': 0.0, 'valid_pixels': 874.0, 'percentile_2': 0.0, 'percentile_98': 12.931278591156005}}}


We can see that some locations near the line are at significant risk of fire. This can help inform where fire mitigation should take place.